In [ ]:
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [ ]:
chole = cv2.imread('/Users/harshpanwar/Desktop/Archive/chole/UNADJUSTEDNONRAW_thumb_2a.jpg')
chole = cv2.cvtColor(chole,cv2.COLOR_BGR2RGB)

In [ ]:
type(chole)

In [ ]:
chole.shape

In [ ]:
plt.imshow(chole)

In [ ]:
rajma = cv2.imread('/Users/harshpanwar/Desktop/Archive/rajma/UNADJUSTEDNONRAW_thumb_2f.jpg')
rajma = cv2.cvtColor(rajma,cv2.COLOR_BGR2RGB)

In [ ]:
rajma.shape

In [ ]:
plt.imshow(rajma)

## Preparing the Data for the model

There is too much data for us to read all at once in memory. We can use some built in functions in Keras to automatically process the data, generate a flow of batches from a directory, and also manipulate the images.

### Image Manipulation

Its usually a good idea to manipulate the images with rotation, resizing, and scaling so the model becomes more robust to different images that our data set doesn't have. We can use the **ImageDataGenerator** to do this automatically for us. Check out the documentation for a full list of all the parameters you can use here!

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [ ]:
plt.imshow(image_gen.random_transform(chole))

In [ ]:
plt.imshow(image_gen.random_transform(chole))

In [ ]:
plt.imshow(image_gen.random_transform(chole))

### Generating many manipulated images from a directory


In order to use .flow_from_directory, you must organize the images in sub-directories. This is an absolute requirement, otherwise the method won't work. The directories should only contain images of one class, so one folder per class of images.

Structure Needed:

* Image Data Folder
    * Class 1
        * 0.jpg
        * 1.jpg
        * ...
    * Class 2
        * 0.jpg
        * 1.jpg
        * ...
    * ...
    * Class n

In [ ]:
image_gen.flow_from_directory('/Users/harshpanwar/Desktop/Folder/train')

In [ ]:
image_gen.flow_from_directory('/Users/harshpanwar/Desktop/Folder/test')

### Resizing Images

Let's have Keras resize all the images to 150 pixels by 150 pixels once they've been manipulated.

In [ ]:
# width,height,channels
image_shape = (150,150,3)

# Creating the Model

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))


model.add(Dense(9))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Training the Model

In [ ]:
batch_size = 16

train_image_gen = image_gen.flow_from_directory('/Users/harshpanwar/Desktop/Folder/train',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
test_image_gen = image_gen.flow_from_directory('/Users/harshpanwar/Desktop/Folder/test',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
train_image_gen.class_indices

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
results = model.fit_generator(train_image_gen,epochs=31,
                              steps_per_epoch=150,
                              validation_data=test_image_gen,
                             validation_steps=12)

In [ ]:
model.save('cat_dog2.h5')

# Evaluating the Model

In [ ]:
results.history['accuracy']

In [ ]:
plt.plot(results.history['accuracy'])

In [ ]:
model.save('cat_dog_100epochs.h5')

# Predicting on new images

In [ ]:
train_image_gen.class_indices

In [ ]:
import numpy as np
from keras.preprocessing import image

file_1 = '/Users/harshpanwar/Desktop/Archive/mutton_bhuna/UNADJUSTEDNONRAW_thumb_1a.jpg'

predicted_img = image.load_img(file_1, target_size=(150, 150))

predicted_img = image.img_to_array(predicted_img)

predicted_img = np.expand_dims(predicted_img, axis=0)
predicted_img = predicted_img/255

In [ ]:
prediction_prob = model.predict(predicted_img)

In [ ]:
model.predict_classes(predicted_img)

In [ ]:
# Output prediction
print(f'Probability that food image is of mutton_bhuna is: {prediction_prob} ')